# Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type по правилам:

если источник traffic_source равен Yandex или Google, то в source_type ставится organic; для источников paid и email из России ставим ad; для источников paid и email не из России ставим other; все остальные варианты берём из traffic_source без изменений.

In [12]:
import pandas as pd
df = pd.read_csv('visit_log.csv', delimiter=';')
df

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex
...,...,...,...,...,...,...
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid
18935,1550094308,e8cf2eb8e6,https://host.ru/a5dda93e70318570c0,Belarus,b85baa8c73,yandex
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid


In [13]:
df['traffic_source'].describe()

count      18938
unique         5
top       direct
freq        7013
Name: traffic_source, dtype: object

In [14]:
def traffic_distribution(line):
  if line['traffic_source'] == 'yandex' or line['traffic_source'] == 'google':
    return 'organic'
  elif line['traffic_source'] == 'paid' or line['traffic_source'] == 'email':
        if line['region'] == 'Russia': return 'ad'
        else: return 'other'
  return line['traffic_source']

In [15]:
%%time
df['source_type'] = df.apply(traffic_distribution, axis=1)
df

Wall time: 262 ms


,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
...,...,...,...,...,...,...,...
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,other
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,ad
18935,1550094308,e8cf2eb8e6,https://host.ru/a5dda93e70318570c0,Belarus,b85baa8c73,yandex,organic
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,ad


In [16]:
df['source_type'].describe()

count      18938
unique         4
top       direct
freq        7013
Name: source_type, dtype: object

In [17]:
df.pivot_table( values='timestamp', index=['region'], columns=['source_type'], aggfunc='count', fill_value= '0.0', margins=True, margins_name='TOTAL')

source_type,ad,direct,organic,other,TOTAL
region,,,,,
Australia,0.0,222.0,179.0,189.0,590
Belarus,0.0,998.0,822.0,937.0,2757
China,0.0,397.0,322.0,390.0,1109
India,0.0,134.0,114.0,121.0,369
Russia,3790.0,4052.0,3071.0,0.0,10913
Ukraine,0.0,1210.0,899.0,1091.0,3200
TOTAL,3790,7013,5407,2728,18938


In [18]:
def traffic_distribution(string):
  lst = string.split(';')
  if 'yandex' in lst[5]  or 'google' in lst[5]:
    lst[5] = lst[5].strip('\n')
    lst.append('organic\n')
  elif 'paid' in lst[5] or 'mail' in lst[5]:
        if lst[3] == 'Russia': 
          lst[5] = lst[5].strip('\n')
          lst.append('ad\n')
        else: 
          lst[5] = lst[5].strip('\n')
          lst.append('other\n')
  else: 
    lst[5] = lst[5].strip('\n')
    lst.append(lst[5]+'\n')
  return lst

In [19]:
%%time
with open('visit_log.csv', 'r') as f:
  with open('plus_traffic.csv', 'w+') as f_append:
    for l in f:
      f_append.write(",".join(traffic_distribution(l)))

Wall time: 37.4 ms


In [20]:
%%time
df = pd.read_csv('plus_traffic.csv',
                 names=['timestamp',	'visit_id',	'url',	'region',	'user_id',	'traffic_source',	'source_type'],
                 skiprows=1)
df

Wall time: 31.3 ms


,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
...,...,...,...,...,...,...,...
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,other
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,ad
18935,1550094308,e8cf2eb8e6,https://host.ru/a5dda93e70318570c0,Belarus,b85baa8c73,yandex,organic
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,ad


# Задание 2
В файле URLs.txt содержатся URL страниц новостного сайта. Вам нужно отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис. Выполните действия:

Прочитайте содержимое файла с датафрейм. Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствие с заданным шаблоном.

In [233]:
import pandas as pd

In [252]:
with open('URLs.txt', 'r') as f:
        df = pd.Series([line for line in f])
        print(df[df.str.contains("/\d{8}-")])

4     /politics/36188461-s-marta-zhizn-rossiyan-susc...
5     /world/36007585-tramp-pridumal-kak-reshit-ukra...
6     /science/36157853-nasa-sobiraet-ekstrennuyu-pr...
7     /video/36001498-poyavilis-pervye-podrobnosti-g...
8     /world/36007585-tramp-pridumal-kak-reshit-ukra...
                            ...                        
89    /cis/35984145-kreml-prokommentiroval-soobschen...
90    /video/36071019-olimpiyskie-obekty-rio-prevrat...
91    /science/36151301-nazvano-posledstvie-zloupotr...
92    /incidents/36027330-vospitatelnitsu-zatravili-...
93    /world/36103095-dominikanskih-zhurnalistov-ubi...
Length: 84, dtype: object


# Задание 3
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значениями столбца timestamp для данного значения userId.

In [452]:
import pandas as pd
df = pd.read_csv('ratings.csv')
df.shape

(100004, 4)

In [453]:
df_group = df.groupby(['userId']).agg({'movieId': 'count', 'timestamp': ['min', 'max']})
df_group.shape

(671, 3)

In [454]:
df_filtred = df_group.loc[df_group[('movieId', 'count')] > 100]
df_filtred.shape

(258, 3)

In [461]:
round(df_filtred.apply(lambda row: row[('timestamp','max')] - row[('timestamp', 'min')], axis=1).mean())

40080507

# Задание 4
Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

Нужно сформировать две таблицы:

*таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;

*аналогичную таблицу по типам выручки с указанием адреса клиента. 


Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [472]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [473]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [474]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [475]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [478]:
with_no_address = pd.merge(pd.merge(rzd, air, how='outer'), auto, how='outer').fillna(0)
with_no_address

,client_id,rzd_revenue,air_revenue,auto_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,0.0,57483.0
3,114,5774.0,0.0,83.0
4,115,981.0,81.0,912.0
5,116,0.0,4.0,4834.0
6,117,0.0,13.0,98.0
7,118,0.0,173.0,0.0


In [483]:
with_address = pd.merge(with_no_address, client_base, how='inner')
with_address

,client_id,rzd_revenue,air_revenue,auto_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,0.0,57483.0,Левобережная 1а
3,114,5774.0,0.0,83.0,Мира 14
4,115,981.0,81.0,912.0,ЗЖБИиДК 1
5,116,0.0,4.0,4834.0,Строителей 18
6,117,0.0,13.0,98.0,Панфиловская 33
7,118,0.0,173.0,0.0,Мастеркова 4
